## Pacotes Necessários

In [1]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests
import time
import datetime
import pandas as pd

In [2]:
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#### Path para o chromedriver

## Coleta de todos anos e etapas

In [ ]:
path = os.path.abspath(os.path.join('..', 'auxiliar'))
path_data = os.path.join(path, 'chromedriver')

In [30]:
driver = webdriver.Chrome(ChromeDriverManager().install())

# Lista de anos
anos = anos = [
    "1984-85", "1985-86", "1986-87", "1987-88", "1988-89",
    "1989-90", "1990-91", "1991-92", "1992-93", "1993-94",
    "1994-95", "1995-96", "1996-97", "1997-98", "1998-99",
    "1999-00", "2000-01", "2001-02", "2002-03", "2003-04",
    "2004-05", "2005-06", "2006-07", "2007-08", "2008-09",
    "2009-10", "2010-11", "2011-12", "2012-13", "2013-14",
    "2014-15", "2015-16", "2016-17", "2017-18", "2018-19"
]

# Lista para armazenar os DataFrames de cada ano
dfs = []


for ano in anos:
    
    # Lista para armazenar os textos
    textos_b = []
    
    nba = 'https://en.wikipedia.org/wiki/'+ ano +'_NBA_season'
    #time.sleep(5)  
    driver.get(nba)
    
    # Aguarda o carregamento da página
    time.sleep(2)

    # Localiza o elemento pela classe 'excerpt-block'
    elemento = driver.find_element(By.CLASS_NAME, "excerpt-block")

    # Faz o scroll até o elemento
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", elemento)

    # Espera para visualizar
    time.sleep(3)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')

    # Encontra todas as divs com a classe 'excerpt-block'
    divs_excerpt = soup.find_all("div", class_="excerpt-block")

    # Itera sobre as divs com a classe 'excerpt-block'
    for div in divs_excerpt:
        # Agora, dentro de cada div, encontramos o <tbody>
        tbody = div.find('tbody')

        if tbody:
            # Pegando todos os <tr> dentro do <tbody>
            trs = tbody.find_all('tr')
            
            for tr in trs:
                # Verifica se o <tr> tem elementos relevantes como <b> ou <a>
                b_elements = tr.find_all('b')
                td_elements = tr.find_all('td', attrs={"rowspan": "2"})  # Buscando por <td> com rowspan="2"

                if b_elements or a_elements or td_elements:  # Se encontrar algo relevante
                    row_data_b = []
                    row_data_td = []

                    # Extrai as informações de <b> e <a> nos <td> relevantes
                    for b in b_elements:
                        row_data_b.append(b.get_text(strip=True))  # Extrai texto de <b>
                      
                    for td in td_elements:
                        row_data_td.append(td.get_text(strip=True))  # Extrai texto de <td> com rowspan="2"

                    # Adiciona as linhas de dados à lista
                    textos_b.append(row_data_b)
                    if row_data_td:  # Adiciona os dados de <td rowspan="2"> separadamente
                        textos_b.append(row_data_td)


    listas = textos_b.copy()
    listas_tratada = [sublista for sublista in listas if len(sublista) > 1]
    listas_filtradas = [listas_tratada[i] for i in range(len(listas_tratada)) if i % 3 != 1]
    dados_filtrados = [list(filter(None, lista))[:3] for lista in listas_filtradas]

    # Criar DataFrame com as colunas 
    df = pd.DataFrame(dados_filtrados, columns=['id', 'name', 'score'])
    df['name'] = df['name'].str.replace('*', '', regex=False)

    # Criando a coluna de disputa
    df['matchup'] = (df.index // 2) + 1

    # Mapeando para as fases
    def etapa_label(disputa):
        if disputa in [1, 3, 5, 7, 9, 11, 13, 15]:
            return "First Round"
        elif disputa in [2, 6, 10, 14]:
            return "Conference Semifinals"
        elif disputa in [4, 12]:
            return "Conference Finals"
        elif disputa == 8:
            return "NBA Finals"

    df['stage'] = df['matchup'].map(etapa_label)
    df['year'] = ano
    
    # Adiciona o DataFrame do ano atual à lista
    dfs.append(df)

    
df_final = pd.concat(dfs, ignore_index=True)
df_final.to_csv('nba_data.csv', index=False)
# Resultado
df_final


<ipython-input-30-440654165ce6>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


,id,name,score,matchup,stage,year
0,E1,Boston,3,1,First Round,1984-85
1,E8,Cleveland,1,1,First Round,1984-85
2,E1,Boston,4,2,Conference Semifinals,1984-85
3,E4,Detroit,2,2,Conference Semifinals,1984-85
4,E4,Detroit,3,3,First Round,1984-85
...,...,...,...,...,...,...
1045,W6,Oklahoma City,1,13,First Round,2018-19
1046,W3,Portland,4,14,Conference Semifinals,2018-19
1047,W2,Denver,3,14,Conference Semifinals,2018-19
1048,W2,Denver,4,15,First Round,2018-19


In [31]:
df_final

,id,name,score,matchup,stage,year
0,E1,Boston,3,1,First Round,1984-85
1,E8,Cleveland,1,1,First Round,1984-85
2,E1,Boston,4,2,Conference Semifinals,1984-85
3,E4,Detroit,2,2,Conference Semifinals,1984-85
4,E4,Detroit,3,3,First Round,1984-85
...,...,...,...,...,...,...
1045,W6,Oklahoma City,1,13,First Round,2018-19
1046,W3,Portland,4,14,Conference Semifinals,2018-19
1047,W2,Denver,3,14,Conference Semifinals,2018-19
1048,W2,Denver,4,15,First Round,2018-19
